In [1]:
import pandas as pd
from scipy.io import arff
from sklearn.metrics import confusion_matrix, cohen_kappa_score
from sklearn.naive_bayes import BernoulliNB

In [2]:
file = "datasets/spambase.arff"

arff_file = arff.loadarff(file)
data = pd.DataFrame(arff_file[0])
# Decode byte strings to regular strings
data = data.map(lambda x: x.decode() if isinstance(x, bytes) else x)
data

,word_freq_make,word_freq_address,word_freq_all,word_freq_3d,word_freq_our,word_freq_over,word_freq_remove,word_freq_internet,word_freq_order,word_freq_mail,...,char_freq_;,char_freq_(,char_freq_[,char_freq_!,char_freq_$,char_freq_#,capital_run_length_average,capital_run_length_longest,capital_run_length_total,Class
0,0.00,0.64,0.64,0.0,0.32,0.00,0.00,0.00,0.00,0.00,...,0.000,0.000,0.0,0.778,0.000,0.000,3.756,61.0,278.0,1
1,0.21,0.28,0.50,0.0,0.14,0.28,0.21,0.07,0.00,0.94,...,0.000,0.132,0.0,0.372,0.180,0.048,5.114,101.0,1028.0,1
2,0.06,0.00,0.71,0.0,1.23,0.19,0.19,0.12,0.64,0.25,...,0.010,0.143,0.0,0.276,0.184,0.010,9.821,485.0,2259.0,1
3,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.000,0.137,0.0,0.137,0.000,0.000,3.537,40.0,191.0,1
4,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.000,0.135,0.0,0.135,0.000,0.000,3.537,40.0,191.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4596,0.31,0.00,0.62,0.0,0.00,0.31,0.00,0.00,0.00,0.00,...,0.000,0.232,0.0,0.000,0.000,0.000,1.142,3.0,88.0,0
4597,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,...,0.000,0.000,0.0,0.353,0.000,0.000,1.555,4.0,14.0,0
4598,0.30,0.00,0.30,0.0,0.00,0.00,0.00,0.00,0.00,0.00,...,0.102,0.718,0.0,0.000,0.000,0.000,1.404,6.0,118.0,0
4599,0.96,0.00,0.00,0.0,0.32,0.00,0.00,0.00,0.00,0.00,...,0.000,0.057,0.0,0.000,0.000,0.000,1.147,5.0,78.0,0


In [3]:
## On retire les 3 attributs qui nous intéressent pas et on binarise

data = data.drop(columns=['capital_run_length_average', 'capital_run_length_longest', 'capital_run_length_total'])
data.iloc[:, :-1] = data.iloc[:, :-1].applymap(lambda x: 0 if x == 0 else 1)

data.columns = [f"{col}_binarized" if col != 'Class' else col for col in data.columns]
data

C:\Users\paull\AppData\Local\Temp\ipykernel_12544\2196348422.py:4: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data.iloc[:, :-1] = data.iloc[:, :-1].applymap(lambda x: 0 if x == 0 else 1)


,word_freq_make_binarized,word_freq_address_binarized,word_freq_all_binarized,word_freq_3d_binarized,word_freq_our_binarized,word_freq_over_binarized,word_freq_remove_binarized,word_freq_internet_binarized,word_freq_order_binarized,word_freq_mail_binarized,...,word_freq_edu_binarized,word_freq_table_binarized,word_freq_conference_binarized,char_freq_;_binarized,char_freq_(_binarized,char_freq_[_binarized,char_freq_!_binarized,char_freq_$_binarized,char_freq_#_binarized,Class
0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1
1,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,1
2,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1
3,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1
4,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4596,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0
4597,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0
4598,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0
4599,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0


On vérifie qu'on a bien les mêmes résultats en faisant la manipulation sur Weka :

![alt text](spam_base_weka.png "Title")

On utilise ici le classifieur BernoulliNB de scikit-learn, qui permet d'utiliser la méthode Bayesienne naïve pour des datasets composés de booléens.
https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.BernoulliNB.html#sklearn.naive_bayes.BernoulliNB

In [4]:
X = data.iloc[:, :-1]
y = data.iloc[:, -1]

gnb = BernoulliNB()
model = gnb.fit(X, y)
y_pred = model.predict(X)


print("Matrice de confusion :\n", confusion_matrix(y, y_pred))
print("Précision :", model.score(X, y))

kappa = cohen_kappa_score(y, y_pred)
print("Kappa :", kappa)

Matrice de confusion :
 [[2598  190]
 [ 335 1478]]
Précision : 0.8858943707889589
Kappa : 0.7576699355221098


***Question 1***

Les résultats données par Weka sont : 
```
    a    b   <-- classified as
 2596  192 |    a = 0
  335 1478 |    b = 1
```
Donc quasiment les mêmes que python.
On a 88.6% d'instances correctement classées, donc le modèle est globalement bon. On a 4.2% de faux positifs, c'est-à-dire des mail non spam qui ont été classifiée comme spam, qui peut être un peu problématique.

***Question 2***

En observant en détail la base de données sur Weka, on remarque qu'il y a des caractèristiques discriminantes dans la base de données tel que "word_freq_lab", "word_freq_telnet", "word_freq_1999", "word_freq_hp". Comme il y a beaucoup d'attributs, cela permet faire la distinction entre les spam et non-spam efficacement. La base de données contient un nombre important d'entrée, ce qui permet au modèle d'avoir de meilleurs résultats.
De plus, dans la méthode bayesienne naïve, on fait l'hypothèse que les attributs sont indépendants entre eux. Si le modèle possède de bon résultats, ce signifie que les différents attributs de la base de données sont réellements indépendants entre eux.

***Question 3***

Si on ne faisait pas l'hypothèse de l'indépendance des attributs sur la base de données, plusieurs problème pourraient survenir, comme la dépendance entre les mots, ou bien la perte d'information contextuelle avec le non-prise en compte de l'ordre des mots. Un autre problème qui peut survenir et le sur-ajustement (trop de paramètres à estimer par rapport à la taille des données) ou sous-ajustument (le modèle devient trop général et perd en précision).

***Question 4***

Pour faire le calcul a priori de chaque classe, Weka calcule juste la fréquence de chaque attribut, et en faisant si nécéssaire un lissage de Laplace.

***Question 5***

In [5]:
attribute = "word_freq_3d_binarized"

spam_data = data[data['Class'] == '1']
no_spam_data = data[data['Class'] == '0']

p_3d_spam = spam_data[attribute].sum()/len(spam_data)
p_3d_no_spam = no_spam_data[attribute].sum()/len(no_spam_data)

print("P(3d|spam) :", p_3d_spam)
print("P(3d|no spam) :", p_3d_no_spam)

P(3d|spam) : 0.02151130722559294
P(3d|no spam) : 0.0028694404591104736


***Question 6***

In [6]:
file_test = "datasets/spambase_test_bool_sans_3_att.arff"

arff_file = arff.loadarff(file_test)
data_test = pd.DataFrame(arff_file[0])
# Decode byte strings to regular strings
data_test = data_test.map(lambda x: x.decode() if isinstance(x, bytes) else x)
data_test

,word_freq_make_binarized,word_freq_address_binarized,word_freq_all_binarized,word_freq_3d_binarized,word_freq_our_binarized,word_freq_over_binarized,word_freq_remove_binarized,word_freq_internet_binarized,word_freq_order_binarized,word_freq_mail_binarized,...,word_freq_edu_binarized,word_freq_table_binarized,word_freq_conference_binarized,char_freq_;_binarized,char_freq_(_binarized,char_freq_[_binarized,char_freq_!_binarized,char_freq_$_binarized,char_freq_#_binarized,Class
0,0,1,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1


In [7]:
X_test = data_test.iloc[:, :-1]
y_test = data_test.iloc[:, -1]

y_pred = model.predict(X_test)
print("Classe de la nouvelle instance :", y_test.values[0])
print("Classe prédite :", y_pred[0])

Classe de la nouvelle instance : 1
Classe prédite : 1


La nouvelle instance est correctement classée. On a le même résultat sur Weka.

In [8]:
non_spam_words = ["hp", "hpl", "george", "650","lab", "labs", "telnet", "857", "415", "85", "technology", "1999", "cs", "meeting", "original", "project", "conference"]
# "pm" et "data" ne sont pas présents dans la liste car ces mots sont déjà présent dans la nouvelle instance
for word in non_spam_words:
    print(f"{word} : {X_test["word_freq_"+word+"_binarized"].values[0]}")

hp : 0
hpl : 0
george : 0
650 : 0
lab : 0
labs : 0
telnet : 0
857 : 0
415 : 0
85 : 0
technology : 0
1999 : 0
cs : 0
meeting : 0
original : 0
project : 0
conference : 0


In [9]:
import random
random.shuffle(non_spam_words)

# On reset la nouvelle instance afin de pouvoir réexécuter directement la cellule si on veut faire un autre test
for word in non_spam_words:
    X_test["word_freq_"+word+"_binarized"].values[0] = 0

for word in non_spam_words:
    X_test["word_freq_"+word+"_binarized"].values[0] = 1
    y_pred = model.predict(X_test)
    print(f"Classe prédite pour {word} :", y_pred[0])

X_test["word_freq_hp_binarized"].values[0] = 1
y_pred = model.predict(X_test)
print(f"Classe prédite pour {"hp"} :", y_pred[0])

Classe prédite pour technology : 1
Classe prédite pour george : 0
Classe prédite pour project : 0
Classe prédite pour labs : 0
Classe prédite pour hpl : 0
Classe prédite pour 650 : 0
Classe prédite pour 1999 : 0
Classe prédite pour telnet : 0
Classe prédite pour 415 : 0
Classe prédite pour meeting : 0
Classe prédite pour 85 : 0
Classe prédite pour cs : 0
Classe prédite pour hp : 0
Classe prédite pour conference : 0
Classe prédite pour 857 : 0
Classe prédite pour lab : 0
Classe prédite pour original : 0
Classe prédite pour hp : 0


Il suffit d'ajouter très peu de mots non spam (1 ou 2 suffisent) pour que la prédiction du classifieur change.